In [1]:
import BioSimSpace.Sandpit.Exscientia as BSS
import numpy as np
import pandas as pd

/home/yanfz/miniforge3/envs/a3fe/lib/python3.12/site-packages/nglview/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
INFO:rdkit:Enabling RDKit 2024.03.5 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


In [2]:
system = BSS.IO.readMolecules(["bound/ensemble_equilibration_1/bound_preequil.prm7","bound/ensemble_equilibration_1/bound_preequil.rst7"])

In [3]:
system[0].getResidues()

[<BioSimSpace.Residue: name='LIG', molecule=35, index=0, nAtoms=97>]

In [4]:
lig = BSS.Align.decouple(system[0])
system.updateMolecule(0, lig)

In [5]:
traj = BSS.Trajectory.Trajectory(
    trajectory="bound/ensemble_equilibration_1/gromacs.xtc",
    topology="bound/ensemble_equilibration_1/bound_preequil.prm7",
)

Indexing XTC               6.2 s 200350976.0 bytes / s          


In [6]:
restraint = BSS.FreeEnergy.RestraintSearch.analyse(
    work_dir = "./get_restraints",
    system = system,
    traj = traj,
    temperature = 298 * BSS.Units.Temperature.kelvin,
    method="BSS",
    restraint_type = "multiple_distance",
)

INFO:MDAnalysis.core.universe:The attribute(s) masses, types have already been read from the topology file. The guesser will only guess empty values for this attribute, if any exists. To overwrite it by completely guessed values, you can pass the attribute to the force_guess parameter instead of the to_guess one
INFO:MDAnalysis.guesser.base:There is no empty types values. Guesser did not guess any new values for types attribute
INFO:MDAnalysis.guesser.base:There is no empty masses values. Guesser did not guess any new values for masses attribute


Searching for low variance pairs. Frame no:   0%|          | 0/2501 [00:00<?, ?it/s]

In [7]:
restrain_dict=restraint.toString(engine="Somd")

In [8]:
restrain_dict

'distance restraints dictionary = {(310, 16): (5.723181318757772, 20.0, 0.8789965279822596), (288, 17): (8.499300605713174, 20.0, 0.9068467390548669), (3220, 47): (5.981220462458057, 20.0, 0.9659520807721043), (6005, 30): (6.4003822268618515, 20.0, 1.037544469912362), (3232, 52): (6.993210694405135, 20.0, 0.9714423776338137), (4119, 56): (5.615441320752831, 20.0, 1.0305605348746898), (3232, 51): (6.625688095737994, 20.0, 1.0108169702282943), (3194, 6): (8.566333712760866, 20.0, 1.0462714335630512), (3308, 49): (5.514469851754981, 20.0, 1.0493017491540453), (4119, 57): (4.689153301433507, 20.0, 1.08460314960816), (6007, 29): (7.293929536045853, 20.0, 1.1510490072313697), (6078, 12): (5.170602160122064, 20.0, 1.0964475318488285), (6078, 13): (5.136500482291785, 20.0, 1.0758787277476403), (6078, 4): (5.871754677258629, 20.0, 1.1163078358614014), (6078, 7): (6.099899369275629, 20.0, 1.1402187278474702), (6105, 2): (9.134269620261271, 20.0, 1.122487857689837), (6078, 8): (6.828905362799864,

In [9]:
restraint.getCorrection(method = "numerical")

/home/yanfz/miniforge3/envs/a3fe/lib/python3.12/site-packages/BioSimSpace/Sandpit/Exscientia/FreeEnergy/_restraint.py:1157: UserWarning: The multiple distance restraint correction is assumes that only the 'permanent_distance_restraint' is active.
  _warnings.warn(


0.0698 kcal/mol

In [11]:
lam_vals_release_restraint = [0.000, 0.100, 0.200, 0.300, 0.400, 0.500, 0.600, 0.700, 0.800, 0.900, 1.000]


release_restraints_protocol = BSS.Protocol.FreeEnergy(runtime=1*BSS.Units.Time.nanosecond, 
                                                      timestep=4*BSS.Units.Time.femtosecond, 
                                                      lam_vals=lam_vals_release_restraint, 
                                                      perturbation_type="release_restraint")

release_restraints_fe_calc = BSS.FreeEnergy.AlchemicalFreeEnergy(restraint.system, 
                                                                 release_restraints_protocol, 
                                                                 engine='somd', 
                                                                 restraint=restraint,
                                                                 work_dir='./get_restraints/release1')

/home/yanfz/miniforge3/envs/a3fe/lib/python3.12/site-packages/BioSimSpace/Sandpit/Exscientia/FreeEnergy/_alchemical_free_energy.py:370: UserWarning: SOMD simulations with some non-interacting ligands can be unstable. This affects some systems with multiple distance restraints during the release restraint state. If you experience problems, consider using multiple distance restraints with GROMACS instead.
  _warnings.warn(
